In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, date

## Group Assignment
### Team Number: 11
### Team Member Names: Jeffrey, Ray, Baldeep
### Team Strategy Chosen: RISKY (RISKY OR SAFE)

# Initialization of Variables

In [2]:
# List of Tickers to Choose From
ticker_list = pd.read_csv("Tickers.csv")

# Initial Capital (CAD)
capital = 750000

# Trading Fee
trading_fee = 4.95

# Restrictions
avg_month_vol = 150000
min_month_days = 18
currency = ['USD', 'CAD']

# Number of stocks in our Portfolio
num_stocks = 10

# Minimum weight for each stock in the portfolio (in decimals)
min_weight = 1 / (2 * num_stocks)

# Maximum weight for each stock in the portfolio (in decimals)
max_weight = 0.2

# Day that we will purchase stocks for our portfolio
purchase_date = '2023-11-25'

In [3]:
# Portfolios
Portfolio_Final = pd.DataFrame()
Stocks_Final = pd.DataFrame()

# Filtering the tickers

## Before creating our portfolio of stocks, we must first create a filter function to filter out unwanted tickers for this assignment. 

We have to consider:
1) Are the stocks delisted?
2) Are the stocks denominated in USD or CAD?
3) Do the months have at least 18 trading days?
4) Do the stocks have an average monthly volume of at least 150 000 shares? 


1) To do this, we loop through every ticker in the ticker_lst and check if calling fast_info produces an error. If it produces an error, we know the stock is delisted, so we don't add that ticker to the new ticker list. 

2) We get the fast_info of the stock and check if the currency is either USD or CAD. If it is, we add it to the new ticker list. 

3) We use a stock index from the TSX and another from the NYSE and concat the historical data together such that only the days that appear in both are kept. Then, we loop through every month and count the total number of days. If the number of days is less than 18, then we add it to a list and remove the month from the dataframes later. 

4) Using the historical data, we calculate the average monthly volume for each stock by looping through all the valid stocks and then using the mean function. If the average monthly volume is less than 150 000, then we remove the stock from the stock list. 

In [4]:
#define start date and end date variables
start_date = '2023-01-01'
end_date = '2023-10-31'

#define holder variables
stock_hist_data = []
invalid_months = []

#produce a list of all the tickers
ticker_lst = ticker_list[ticker_list.columns[0]]

#filter stocks variable 
def filter_stocks(ticker_lst):
    new_ticker_lst=[]
    
    #loop through every ticker in the ticker_lst and check if it is listed and has a currency of USD or CAD
    for cur_ticker in ticker_lst:
        info = yf.Ticker(cur_ticker).fast_info
        try:
            #check if the currency is USD or CAD
            if(info.currency == 'USD' or info.currency == 'CAD'):
                new_ticker_lst.append(cur_ticker)
        except:
            #output a statement if the stock is delisted
            print(f"{cur_ticker} is not a valid ticker")
    
    #get the historical data of each ticker and add it to a list
    for cur_ticker in new_ticker_lst:
        temp_ticker = yf.Ticker(cur_ticker)
        hist = temp_ticker.history(start=start_date, end=end_date, interval = '1mo')
        stock_hist_data.append(hist)
    
    cad_index = '^GSPTSE' #S&P/TSX Composite Index (CAD)
    usa_index = '^GSPC' #S&P500 Composite Index (USD)
    
    #loop through every month and check if the month has less than 18 trading days
    for cur_month in range(1, 11):
        #get the historical dataframes of the two indexes
        cad_days = yf.Ticker(cad_index).history(start = str(date(2023, cur_month, 1)), end = str(date(2023, cur_month+1, 1)))
        usd_days = yf.Ticker(usa_index).history(start = str(date(2023, cur_month, 1)), end = str(date(2023, cur_month+1, 1)))
        #merge the two dataframes and only keep the duplicates
        total_days = cad_days.reindex(usd_days.index)
        num_days = len(total_days)
        if num_days < min_month_days:
            invalid_months.append(cur_month)
            
    #remove the months that have less than 18 trading days
    for i in range(len(stock_hist_data)):
        cur_hist = stock_hist_data[i]
        for month in invalid_months:
            if month < 10:
                cur_hist.filter(like!=f'2023-0{month}', axis=0)
            else:
                cur_hist.filter(like != f'2023-{month}', axis=0)
    #Calculate the average monthly volume for each stock and remove the stock if it is less than 150 000
    for i in range(len(new_ticker_lst)): 
        cur_ticker = new_ticker_lst[i]
        cur_hist = stock_hist_data[i]
        avgMonthlyVolume = cur_hist['Volume'].mean()
        #check if the average monthly volume is less than 150 000 and remove the stock if it is
        if avgMonthlyVolume < avg_month_vol:
            new_ticker_lst.remove(cur_ticker)
            stock_hist_data.remove(cur_hist)
    return new_ticker_lst

#call the function
filtered_ticker_lst = filter_stocks(ticker_lst)
filtered_ticker_lst

AGN is not a valid ticker
CELG is not a valid ticker
MON is not a valid ticker
RTN is not a valid ticker


['ABBV',
 'ABT',
 'ACN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CL',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MRK',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RY.TO',
 'SHOP.TO',
 'T.TO',
 'TD.TO',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

# Computing Historical Data

In [5]:
#define start date and end date
compute_start_date = '2022-01-01'
compute_end_date = '2023-11-21'

#get the exchange rate for the compute start date
cad_usd = yf.Ticker('CADUSD=X')
cad_usd_close = cad_usd.history(start=compute_start_date, end=compute_end_date).Close
exchange_rate = cad_usd_close.iloc[0]

weekly_pct_change = pd.DataFrame()

#loop through each ticker and add the percent change of the weekly closing price to the dataframe
for ticker in filtered_ticker_lst:
    hist = yf.Ticker(ticker).history(start=compute_start_date, end=compute_end_date, interval = '1wk')
    temp_currency = yf.Ticker(ticker).fast_info['currency']
    #convert the USD close prices to CAD
    if temp_currency == 'USD':
        hist['Close'] = hist['Close']/exchange_rate
    weekly_pct_change[ticker] = hist.Close.pct_change()

#drop the NaN values
weekly_pct_change = weekly_pct_change.dropna()

#output the weekly percent changes of each stock
weekly_pct_change

    


,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BIIB,BK,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2022-01-08 00:00:00-05:00,0.007340,-0.054588,-0.046905,0.019262,-0.002559,-0.031869,0.048538,-0.025823,0.028805,0.020375,...,0.045896,0.039083,-0.044331,0.005394,0.032304,0.042187,0.022002,-0.033401,-0.053264,0.036885
2022-01-15 00:00:00-05:00,-0.018551,-0.014717,-0.047713,-0.074140,-0.120237,-0.057363,-0.090813,-0.062409,-0.078479,-0.100943,...,-0.125921,-0.017924,-0.196567,-0.014085,-0.021081,-0.060799,-0.016045,0.000244,-0.022223,-0.128854
2022-01-22 00:00:00-05:00,0.045007,-0.003815,0.021512,-0.006281,0.009359,0.115409,-0.072381,0.021149,0.021268,0.022735,...,0.011399,-0.010163,0.003224,0.016667,-0.000897,0.009394,0.010603,-0.001624,-0.018321,0.054084
2022-01-29 00:00:00-05:00,0.019794,0.034783,0.013563,0.070751,0.094886,0.049644,0.083329,0.052540,-0.016340,0.063135,...,0.075895,0.034432,0.003824,0.031114,0.053987,-0.026707,0.036712,-0.014394,0.133871,0.027893
2022-02-05 00:00:00-05:00,0.009669,-0.032534,-0.052719,-0.004099,-0.027569,0.032069,0.028336,-0.007457,-0.031328,0.008413,...,-0.082632,-0.004221,-0.025656,0.024335,0.020735,-0.040952,-0.010431,-0.014770,-0.050269,-0.019263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-21 00:00:00-04:00,-0.049921,-0.040608,-0.023434,0.000168,0.020532,-0.001837,-0.001944,-0.043330,-0.089066,-0.000720,...,-0.019976,-0.022881,-0.082026,-0.004973,-0.032397,-0.031730,-0.004497,-0.045519,-0.112727,-0.009376
2023-10-28 00:00:00-04:00,0.017923,0.032202,0.080851,0.070721,0.085017,0.081028,0.085481,0.129122,0.063875,0.097976,...,0.122675,0.083128,0.306294,0.101772,0.063288,0.049679,0.011894,0.051606,0.059556,0.158290
2023-11-04 00:00:00-04:00,-0.020011,-0.019512,0.019777,0.001255,0.035786,0.010605,0.008203,-0.026038,-0.099038,0.009061,...,0.039240,0.006290,0.007257,-0.018969,0.012102,-0.011255,0.019420,-0.003017,-0.033249,-0.036348


## Standard Deviation


In creating a risky portfolio, we require a group of stock tickers that are unpredictable and have a high varience when compared to their mean values. These tickers would have large periods of unprecedented growhth and loss, and would be an investors greatest dream come true, or worst nightmare, depending on the performance ot the stock. For the purposes of our portfolio, we are calculating the standard deviation of all of the filtered tickers, and moving forward with the 10 tickers with the highest standard deviations, as this will make up our portfolio of risky, volatile stocks. 

\begin{align*}
\sigma_X=\sqrt{\frac{\sum(x_i-\overline{X})^2}{N}}
\end{align*}

In [6]:
# Function to return the tickers of the firms with the ten highest standard deviations in the original ticker list
def highest_std(dataframe):
    
    # Setting variables
    temp_std = 0 
    list_of_stds = []
    list_of_ten_tickers = []
    
    # Looping through every column in the datafarme
    for x in dataframe.columns:
        temp_std = abs(dataframe[x].std())
        list_of_stds.append(temp_std)
    
    # Return a dataframe with the list of standard deviations
    global stds_dataframe
    stds_dataframe = pd.DataFrame(list_of_stds, index=dataframe.columns, columns=['std'])
    stds_dataframe.sort_values("std", inplace=True, ascending=False)
    
    for x in range (10):
        temp_ticker = (stds_dataframe.index[x])
        list_of_ten_tickers.append(temp_ticker)
        
    
    return list_of_ten_tickers

print(highest_std(weekly_pct_change))

['SHOP.TO', 'PYPL', 'BA', 'AMZN', 'QCOM', 'USB', 'BIIB', 'CAT', 'BLK', 'AXP']


## Correlation and Covarience

Since we have 10 tickers, then the minimum weight for each is 5% and the max is 20%. We want highest variance and
volatility of the stocks (risky strategy). So, loop through the 10 tickers found above with the highest standard
deviation, and for each (call this stock the base stock), find the 9 stocks that have the highest correlation to to
the base stock in decreasing order. (meaning that the first stock will have the highest correlation
to the base stock, the second stock will have the second highest correlation to the base stock, and so on.) Then,
put 20% in the base stock, the first stock, the second stock, 10% in the third stock, and 5% in the rest in order
to maximize the expected combined correlation (weight * correlation)

In [7]:
# To find which portfolio gives the best percent returns
pfl_pct_returns = []
expected_correlation = []

# Made into a function for future use
def correlation_pfl(tkr_lst, return_lst):
    
    '''
    From all the tickers in tkr_lst, finds the next 9 highest correlated stocks and creates a weighted portfolio. Then 
    using historical data from Jan 1, 2022 to Nov 21, 2023, find the portfolio average percent returns and appends the 
    value to the return_lst
    ''' 
    for i in tkr_lst:
        temp_df = pd.DataFrame()
        correlation = weekly_pct_change.corr()[i]
        correlation = correlation.sort_values(ascending=False)
        
        # List of largest correlation values to the base stock
        stock_lst = [k for k in (correlation.index[:10])]

        print(stock_lst)
        
        # To get the portfolio of each and calculate average percent returns
        for x in stock_lst:
            temp_df[x] = weekly_pct_change[x]
        
        # Finding the weighted returns of each stock in the portfolio
        count = 0
        for x in temp_df.columns:
            if count <= 2:
                temp_df[x] = temp_df[x] * 0.2
                count += 1
            elif count == 3:
                temp_df[x] = temp_df[x] * 0.1
                count += 1
            else:
                temp_df[x] = temp_df[x] * 0.05
                count += 1
                
        display(temp_df.head())
        
        # Finding the weighted average portfolio percent returns. Take absolute value of the average portfolio percent
        # returns since all we care about is maximum volatility and risk
        # (Increase or Decrease in Portfolio value does not matter)
        avg = temp_df.values.sum() / len(temp_df.index)
        return_lst.append(abs(avg))
        

# Running the Function
correlation_pfl(highest_std(weekly_pct_change), pfl_pct_returns)
    
# Now find the index of the highest average percent returns and make a portfolio
max_index = pfl_pct_returns.index(max(pfl_pct_returns))
print("-"*75)
print(f"The portfolio with the highest average percent returns is portfolio {max_index+1}")
print("-"*75)

# Make the portfolio with the corresponding base stock
pfl = pd.DataFrame()
base_stock = highest_std(weekly_pct_change)[max_index]
correlation = weekly_pct_change.corr()[base_stock]
correlation = correlation.sort_values(ascending=False)
stock_lst = [k for k in (correlation.index[:10])]

for y in stock_lst:
    pfl[y] = weekly_pct_change[y]
    
display(pfl.head())

['SHOP.TO', 'PYPL', 'BLK', 'AMZN', 'QCOM', 'AXP', 'TXN', 'ACN', 'RY.TO', 'UPS']


,SHOP.TO,PYPL,BLK,AMZN,QCOM,AXP,TXN,ACN,RY.TO,UPS
Date,,,,,,,,,,
2022-01-08 00:00:00-05:00,-0.008866,-0.009787,-0.009789,-0.000256,0.002295,-0.001593,0.002109,-0.002345,0.001954,-0.002663
2022-01-15 00:00:00-05:00,-0.039313,-0.016680,-0.011237,-0.012024,-0.006296,-0.002868,-0.003040,-0.002386,-0.000896,-0.001111
2022-01-22 00:00:00-05:00,0.000645,-0.000024,0.001803,0.000936,0.000570,0.005770,0.000470,0.001076,-0.000508,-0.000916
2022-01-29 00:00:00-05:00,0.000765,-0.045793,0.000416,0.009489,0.003795,0.002482,-0.001335,0.000678,0.001722,0.006694
2022-02-05 00:00:00-05:00,-0.005131,-0.017116,-0.009219,-0.002757,-0.004132,0.001603,-0.002048,-0.002636,-0.000211,-0.002513


['PYPL', 'BLK', 'SHOP.TO', 'TXN', 'ACN', 'AXP', 'QCOM', 'AMZN', 'C', 'ABT']


,PYPL,BLK,SHOP.TO,TXN,ACN,AXP,QCOM,AMZN,C,ABT
Date,,,,,,,,,,
2022-01-08 00:00:00-05:00,-0.009787,-0.009789,-0.008866,0.004219,-0.002345,-0.001593,0.002295,-0.000128,0.000874,-0.002729
2022-01-15 00:00:00-05:00,-0.016680,-0.011237,-0.039313,-0.006080,-0.002386,-0.002868,-0.006296,-0.006012,-0.002734,-0.000736
2022-01-22 00:00:00-05:00,-0.000024,0.001803,0.000645,0.000939,0.001076,0.005770,0.000570,0.000468,0.001122,-0.000191
2022-01-29 00:00:00-05:00,-0.045793,0.000416,0.000765,-0.002671,0.000678,0.002482,0.003795,0.004744,0.001074,0.001739
2022-02-05 00:00:00-05:00,-0.017116,-0.009219,-0.005131,-0.004095,-0.002636,0.001603,-0.004132,-0.001378,0.001210,-0.001627


['BA', 'C', 'BAC', 'AIG', 'BK', 'BLK', 'UPS', 'AXP', 'USB', 'AMZN']


...

,BLK,ACN,BAC,TXN,AXP,C,PYPL,UPS,BK,SHOP.TO
Date,,,,,,,,,,
2022-01-08 00:00:00-05:00,-0.009789,-0.009381,-0.005165,0.004219,-0.001593,0.000874,-0.002447,-0.002663,0.001019,-0.002217
2022-01-15 00:00:00-05:00,-0.011237,-0.009543,-0.012482,-0.006080,-0.002868,-0.002734,-0.004170,-0.001111,-0.005047,-0.009828
2022-01-22 00:00:00-05:00,0.001803,0.004302,0.004230,0.000939,0.005770,0.001122,-0.000006,-0.000916,0.001137,0.000161
2022-01-29 00:00:00-05:00,0.000416,0.002713,0.010508,-0.002671,0.002482,0.001074,-0.011448,0.006694,0.003157,0.000191
2022-02-05 00:00:00-05:00,-0.009219,-0.010544,-0.001491,-0.004095,0.001603,0.001210,-0.004279,-0.002513,0.000421,-0.001283


['AXP', 'BAC', 'BLK', 'C', 'USB', 'BK', 'AIG', 'ACN', 'QCOM', 'TD.TO']


,AXP,BAC,BLK,C,USB,BK,AIG,ACN,QCOM,TD.TO
Date,,,,,,,,,,
2022-01-08 00:00:00-05:00,-0.006374,-0.005165,-0.009789,0.001748,0.001844,0.001019,0.000963,-0.002345,0.002295,0.001615
2022-01-15 00:00:00-05:00,-0.011473,-0.012482,-0.011237,-0.005468,-0.006443,-0.005047,-0.003707,-0.002386,-0.006296,-0.001054
2022-01-22 00:00:00-05:00,0.023082,0.004230,0.001803,0.002244,0.002704,0.001137,-0.000314,0.001076,0.000570,-0.000045
2022-01-29 00:00:00-05:00,0.009929,0.010508,0.000416,0.002149,0.001395,0.003157,0.003538,0.000678,0.003795,0.002699
2022-02-05 00:00:00-05:00,0.006414,-0.001491,-0.009219,0.002420,-0.000963,0.000421,-0.000205,-0.002636,-0.004132,0.001037


---------------------------------------------------------------------------
The portfolio with the highest average percent returns is portfolio 2
---------------------------------------------------------------------------


,PYPL,BLK,SHOP.TO,TXN,ACN,AXP,QCOM,AMZN,C,ABT
Date,,,,,,,,,,
2022-01-08 00:00:00-05:00,-0.048934,-0.048943,-0.044331,0.042187,-0.046905,-0.031869,0.045896,-0.002559,0.017483,-0.054588
2022-01-15 00:00:00-05:00,-0.083399,-0.056187,-0.196567,-0.060799,-0.047713,-0.057363,-0.125921,-0.120237,-0.054684,-0.014717
2022-01-22 00:00:00-05:00,-0.000122,0.009015,0.003224,0.009394,0.021512,0.115409,0.011399,0.009359,0.022444,-0.003815
2022-01-29 00:00:00-05:00,-0.228963,0.002079,0.003824,-0.026707,0.013563,0.049644,0.075895,0.094886,0.021487,0.034783
2022-02-05 00:00:00-05:00,-0.085581,-0.046097,-0.025656,-0.040952,-0.052719,0.032069,-0.082632,-0.027569,0.024196,-0.032534


# Stock Allocation

In [8]:
top_std_stocks = highest_std(weekly_pct_change)

top_corr_stocks = []

for ticker in top_std_stocks:
    temp_df2 = weekly_pct_change.copy(deep=True)
    correlation2 = weekly_pct_change.corr()[ticker]
    correlation2 = correlation2.sort_values(ascending=False)
    for k in range(10, correlation2.size):
        correlation2 = correlation2.drop(correlation2.index[10])
    correlation2 = correlation2.drop(correlation2.index[0])
    top_corr_stocks.append(correlation2)

top_corr_stocks
    

[PYPL     0.621747
 BLK      0.617827
 AMZN     0.546071
 QCOM     0.538748
 AXP      0.523173
 TXN      0.509658
 ACN      0.508259
 RY.TO    0.475640
 UPS      0.471083
 Name: SHOP.TO, dtype: float64,
 BLK        0.661386
 SHOP.TO    0.621747
 TXN        0.585681
 ACN        0.539638
 AXP        0.538226
 QCOM       0.524136
 AMZN       0.466153
 C          0.461843
 ABT        0.429129
 Name: PYPL, dtype: float64,
 C       0.606688
 BAC     0.576859
 AIG     0.574082
 BK      0.556572
 BLK     0.556219
 UPS     0.511713
 AXP     0.506588
 USB     0.483504
 AMZN    0.477427
 Name: BA, dtype: float64,
 ACN        0.604897
 BLK        0.597770
 TXN        0.562830
 AXP        0.559218
 SHOP.TO    0.546071
 QCOM       0.531607
 UPS        0.526562
 BA         0.477427
 PYPL       0.466153
 Name: AMZN, dtype: float64,
 TXN        0.626734
 AXP        0.602283
 BLK        0.585722
 ACN        0.566268
 C          0.552427
 SHOP.TO    0.538748
 BAC        0.533085
 UPS        0.532027
 AMZ

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.